# Text Summarising of the articles

In [1]:
# Importing the required Libraries
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt') # one time execution
import re
#nltk.download('stopwords') # one time execution
import matplotlib.pyplot as plt

from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx

In [2]:
# Extract word vectors
word_embeddings = {}
file = open('glove.6B.100d.txt', encoding='utf-8')
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
file.close()
len(word_embeddings)

FileNotFoundError: ignored

In [3]:
# reading the file
df = pd.read_excel('TASK.xlsx')

In [4]:
df

,TEST DATASET,Unnamed: 1
0,NaN,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [5]:
df.columns

Index(['TEST DATASET', 'Unnamed: 1'], dtype='object')

In [6]:
df.rename(columns = {'Unnamed: 1' : 'Introduction' }, inplace=True)
# Deleting the first row
df.drop(0)

,TEST DATASET,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
5,NaN,Alkasol Oral Solution is a medicine used in th...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [7]:
# Converting the DataFrame into a dictionary
text_dictionary = {}
for i in range(1,len(df['TEST DATASET'])):
    text_dictionary[i] = df['Introduction'][i]
    
print(text_dictionary[1])

Acnesol Gel is an antibiotic that fights bacteria. It is used to treat acne, which appears as spots or pimples on your face, chest or back. This medicine works by attacking the bacteria that cause these pimples.Acnesol Gel is only meant for external use and should be used as advised by your doctor. You should normally wash and dry the affected area before applying a thin layer of the medicine. It should not be applied to broken or damaged skin. Avoid any contact with your eyes, nose, or mouth. Rinse it off with water if you accidentally get it in these areas. It may take several weeks for your symptoms to improve, but you should keep using this medicine regularly. Do not stop using it as soon as your acne starts to get better. Ask your doctor when you should stop treatment.Common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people. These are usually temporary and resolve on their own. Consult your doctor if they bother you or do not

**There are 1000 such description of the different medicines. The task is to give summarised form of these description.**

In [8]:
# function to remove stopwords
def remove_stopwords(sen):
    stop_words = stopwords.words('english')
    
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [9]:
# function to make vectors out of the sentences
def sentence_vector_func (sentences_cleaned) : 
    sentence_vector = []
    for i in sentences_cleaned:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vector.append(v)
    
    return (sentence_vector)

In [10]:
# function to get the summary of the articles
# NOTE - Remove '#' infront of print statement for displaying the contents at different stages of the text summarisation process
def summary_text (test_text, n = 5):
    sentences = []
    
    # tokenising the text 
    sentences.append(sent_tokenize(test_text))
    # print(sentences)
    sentences = [y for x in sentences for y in x] # flatten list
    # print(sentences)
    
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    #print(clean_sentences)

    
    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    #print(clean_sentences)
    
    sentence_vectors = sentence_vector_func(clean_sentences)
    
    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    #print(sim_mat)
    
    # Finding the similarities between the sentences 
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    #print(scores)
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)))
    # Extract sentences as the summary
    summarised_string = ''
    for i in range(n):
        
        try:
            summarised_string = summarised_string + str(ranked_sentences[i][1])            
        except IndexError:
            print ("Summary Not Available")
    
    return (summarised_string)

In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:

print("Kindly let me know in how many sentences you want the summary - ")
x = int(input())

summary_dictionary = {}

for key in text_dictionary:
    
    para = text_dictionary[key]
    print("Summary of the article - ",key)
    summary = summary_text(para,x)
    summary_dictionary[key] = summary
    
    print(summary)
    print('='*120)    
    
print ("*"*40,"The process has been completed successfully","*"*40)

Kindly let me know in how many sentences you want the summary - 
2
Summary of the article -  1
Acnesol Gel is an antibiotic that fights bacteria.Also, inform the doctor if you have ever had bloody diarrhea caused by taking antibiotics or if you are using any other medicines to treat skin conditions.
Summary of the article -  2
Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus.For better results, it is suggested to take it at the same time every day.
Summary of the article -  3
Augmentin 625 Duo Tablet is a penicillin-type of antibiotic that helps your body fight infections caused by bacteria.Do not stop taking it until you have finished, even when you feel better.
Summary of the article -  4
Azithral 500 Tablet is an antibiotic used to treat various types of bacterial infections of the respiratory tract, ear, nose, throat, lungs, skin, and eye in adults and children.Consult your doctor if you find these side effects worry you or per

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will cha

Allegra 120mg Tablet belongs to a group of medicines called antihistamines.Avoid drinking alcohol as it can cause excessive drowsiness with this medicine.
Summary of the article -  7
Ascoril LS Syrup is a combination medicine used in the treatment of cough with mucus.Avoid drinking alcohol while taking this medicine as it can make sleepiness worse.Never support self-medication or recommend your medicine to another person.
Summary of the article -  8
Aciloc 150 Tablet is a medicine that reduces the amount of excess acid make by your stomach.Also tell your doctor what other medicines you are taking as some may affect, or be affected by, this medicine.
Summary of the article -  9
Avil 25 Tablet is an antiallergic medication used in the treatment of various allergic conditions.Avoid driving or attention-seeking activity.
Summary of the article -  10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


Azee 500 Tablet is an antibiotic used to treat various types of bacterial infections of the respiratory tract, ear, nose, throat, lungs, skin, and eye in adults and children.Consult your doctor if you find these side effects worry you or persist for a longer duration.Inform your doctor if you have any previous history of allergy or heart problems before taking this medicine.
Summary of the article -  11
Aciloc RD 20 Tablet is a prescription medicine used to treat gastroesophageal reflux disease (Acid reflux) and peptic ulcer disease by relieving the symptoms such as heartburn, stomach pain, or irritation.Avoid drinking alcohol while taking this medicine as it can worsen your dizziness.
Summary of the article -  12
Arcolane 2% Scalp Solution belongs to a group of medicines called antifungals.Avoid direct contact with your eyes.
Summary of the article -  13
Atarax 25mg Tablet is an antihistamine used to treat symptoms of skin allergy like itching, swelling, and rashes.Generally, it is ad

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


Also, pregnant women and breastfeeding mothers should not take this medicine as it may harm the developing baby.Atorec 10 Tablet belongs to a group of medicines called statins.
Summary of the article -  993
And do not stop taking the medicine until your doctor tells you it is okay to stop.Using this medicine may cause few common side effects such as nausea, diarrhea, constipation, indigestion, and heartburn.Art Tablet is a combination of three medicines.
Summary of the article -  994
Axaria 10mg Tablet is an antihistamine used to treat symptoms of skin allergy like itching, swelling, and rashes.Generally, it is advised to avoid alcohol while on treatment.
Summary of the article -  995
Additionally, you may notice some injection site reactions like pain, swelling or redness.Ampitrust 1.5gm Injection is a combination of two antibiotics.
Summary of the article -  996


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


Azapure Tablet belongs to a group of medicines called immunosuppressants.Because it suppresses your immune system, you may catch more infections than usual.
Summary of the article -  997
Arimidex 1mg Tablet  is used alone or with other treatments, such as surgery or radiation, to treat early breast cancer in postmenopausal women.If these bother you or appear serious, let your doctor know.
Summary of the article -  998
Also, tell your doctor about all other medicines you are taking as they may affect the working of the medicine.Arpimune ME 100mg Capsule is used to prevent your body rejecting a new organ after a liver, kidney or heart transplant.
Summary of the article -  999
Amlodac CH Tablet is a combination medicine used in the treatment of hypertension (high blood pressure).Consult your doctor If any of these bother you, or get worse, or do not go away.
Summary of the article -  1000
Also, inform your doctor if you have very low blood pressure (hypotension), heart failure, or if you 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


In [17]:
summary_table = pd.DataFrame(list(summary_dictionary.items()),columns = ['TEST DATASET','Summary'])

In [18]:
data_table = pd.DataFrame(list(text_dictionary.items()),columns = ['TEST DATASET','Introduction'])

In [19]:
# Combining the findings into the table
result  = pd.concat([data_table , summary_table['Summary']], axis = 1 , sort = False)
result

,TEST DATASET,Introduction,Summary
0,1,Acnesol Gel is an antibiotic that fights bacte...,Acnesol Gel is an antibiotic that fights bacte...
1,2,Ambrodil Syrup is used for treating various re...,Ambrodil Syrup is used for treating various re...
2,3,Augmentin 625 Duo Tablet is a penicillin-type ...,Augmentin 625 Duo Tablet is a penicillin-type ...
3,4,Azithral 500 Tablet is an antibiotic used to t...,Azithral 500 Tablet is an antibiotic used to t...
4,5,Alkasol Oral Solution is a medicine used in th...,Alkasol Oral Solution is a medicine used in th...
...,...,...,...
995,996,Azapure Tablet belongs to a group of medicines...,Azapure Tablet belongs to a group of medicines...
996,997,Arimidex 1mg Tablet is used alone or with oth...,Arimidex 1mg Tablet is used alone or with oth...
997,998,Arpimune ME 100mg Capsule is used to prevent y...,"Also, tell your doctor about all other medicin..."
998,999,Amlodac CH Tablet is a combination medicine us...,Amlodac CH Tablet is a combination medicine us...


In [ ]:
# Saving it to a file (remove the '#' to save)
#result.to_csv("Summary_File.csv")